In [ ]:
import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns 

from sklearn.metrics import silhouette_score, silhouette_samples
import sklearn.metrics
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import AgglomerativeClustering
from sklearn.mixture import GaussianMixture

import itertools

import scipy

#import sys
#!{sys.executable} -m pip install pandas-profiling
import pandas_profiling
#from yellowbrick.cluster import SilhouetteVisualizer, InterclusterDistance, KElbowVisualizer

#from kmodes.kmodes import KModes

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# This will ensure that matplotlib figures don't get cut off when saving with savefig()
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})

# Reading Data and Viewing Data

In [ ]:
data = pd.read_csv("Clustering Casino Players.csv")
data.head(5)

In [ ]:
# Using Pandas profiling to view all the data
#pandas_profiling.ProfileReport(data)

# CLEANING DATA

## Part 1 - Replacing/Imputing missing data

In [ ]:
#Part 1 replacing/imputing missing data
pd.DataFrame(data).isna().sum() # Lets check if we have any missing data -- the result shows that yes, and a lot

from sklearn.impute import SimpleImputer
# Approach:
# Categorical features: adding new category 'missing_value' (as if this is a new color, or gender)
# Numerical features: replacing with a median (or mean, or most frequent value, etc.) An alternative approach is to imputate, see here: https://scikit-learn.org/stable/modules/impute.html 
# + we add surrogate columns indicating that the value is imputed

# creating surrogates
for col in data:
    if data[col].isna().sum() != 0: 
        data[col + '_surrogate'] = data[col].isna().astype(int)


# fixing categoricals
imputer = SimpleImputer(missing_values = np.nan, strategy='constant')
imputer.fit(data.select_dtypes(exclude=['int64','float64']))
data[data.select_dtypes(exclude=['int64','float64']).columns] = imputer.transform(data.select_dtypes(exclude=['int64','float64']))
           
# fixing numericals 
imputer = SimpleImputer(missing_values = np.nan, strategy='median')
imputer.fit(data.select_dtypes(include=['int64','float64']))
data[data.select_dtypes(include=['int64','float64']).columns] = imputer.transform(data.select_dtypes(include=['int64','float64']))

data.head(5)

## Part 2 - Combining categories into "other"

In [ ]:
#Looking at all the rare categories within the City feature
#City feature has the highest number of level
df=data.copy()
df['City'].value_counts()

In [ ]:
#Part 2 for cleaning data - combining categories into "other"

threshold_percent = 0.27
series = pd.value_counts(df['City'])
mask = (series / series.sum() * 100).lt(threshold_percent)
df = df.assign(City = np.where(df['City'].isin(series[mask].index),'Other_city', df['City']))

df['City'].value_counts()

In [ ]:
df=df.drop(['ID','Site','Age'],axis=1)
df.head(5)

# Ratio Calculation

In [ ]:
#Building features - Ratios

df['one_month_theo_NW']=data['One_Month_Theo_NW']/data['Twelve_Month_Theo_NW']
df['three_month_theo_NW']=data['Three_Month_Theo_NW']/data['Twelve_Month_Theo_NW']
df['six_month_theo_NW']=data['Six_Month_Theo_NW']/data['Twelve_Month_Theo_NW']

df['one_month_act_NW']=data['One_Month_Actual_NW']/data['Twelve_Month_Actual_NW']
df['three_month_act_NW']=data['Three_Month_Actual_NW']/data['Twelve_Month_Actual_NW']
df['six_month_act_NW']=data['Six_Month_Actual_NW']/data['Twelve_Month_Actual_NW']

df['one_month_trips']=data['One_Month_Trips']/data['Twelve_Month_Trips']
df['three_month_trips']=data['Three_Month_Trips']/data['Twelve_Month_Trips']
df['six_month_trips']=data['Six_Month_Trips']/data['Twelve_Month_Trips']

df.head(5)

# Scaling

In [ ]:
n_features = df.shape[1]

#separating categirical and numerical variables
cat_col_names = list(df.select_dtypes(include=np.object).columns)
num_col_names = list(df.select_dtypes(include=np.number).columns)

X_num = df[num_col_names].to_numpy()
X_cat = df[cat_col_names].to_numpy()

#Scaling only the numerical variables
scaler = StandardScaler()
X_num = scaler.fit_transform(X_num)

X = np.concatenate((X_num, X_cat), axis=1)
X

# Heirarchial

In [ ]:
#Defining a custom distance function for the heirarchial model to deal with categorical and numerical data
from scipy.spatial import distance

def custom_dist(x1, x2, cat_cols=None, num_cols=None):
   
    n = len(x1)

    dist_num = distance.euclidean(x1[num_cols], x2[num_cols])
    dist_cat = distance.hamming(x1[cat_cols], x2[cat_cols])
    
    dist = (dist_num*sum(num_cols) + dist_cat*sum(cat_cols)) / n
    
    return dist

In [ ]:
#Creating a custom distance matrix across all columns
from scipy.spatial.distance import pdist

cat_cols = [type(x)==str for x in X[1,:]]
num_cols = [not x for x in cat_cols]

Y = pdist(X, custom_dist, cat_cols=cat_cols, num_cols=num_cols)

In [ ]:
# Perform hierarchical clustering, using our custom-built distance matrix.
import scipy.cluster

aggl = scipy.cluster.hierarchy.linkage(Y, method='ward')

In [ ]:
#Plotting the Dendogram
plt.figure(figsize=(16, 10));
plt.grid(False)
dend = scipy.cluster.hierarchy.dendrogram(aggl);

In [ ]:
#Using k-modes we dervied k=5 for clustering
K=7
labels = scipy.cluster.hierarchy.fcluster(aggl, K, criterion="maxclust")

In [ ]:
#Viewing each cluster and its statistics

from scipy import stats
import seaborn as sns

pd.set_option("display.precision", 2)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

print('All Data:')
print('Number of Instances: {}'.format(X.shape[0]))
df.describe(include=[np.number]).transpose()
df.describe(include=[np.object]).transpose()

for col in cat_col_names:
    df[col].value_counts()

for i, label in enumerate(set(labels)):
    n = df.iloc[labels==label].shape[0]
          
    print('\nCluster {}:'.format(label))
    print('Number of Instances: {}'.format(n))

    df.iloc[labels==label].describe(include=[np.number]).transpose()
    df.iloc[labels==label].describe(include=[np.object]).transpose()
    
    for col in cat_col_names:
        df.iloc[labels==label][col].value_counts()

In [ ]:
X_tmp=df.copy()
X_tmp['Cluster ID']=labels
X_tmp

In [ ]:
X_tmp.to_csv("ClusterID.csv")